In [4]:
#shor algorithm working code
#it's tested on qiskit 1.2.4

#this code is based on https://www.youtube.com/watch?v=-zNUHGcyDd8&t=3s  
#   from joonionTV(https://www.youtube.com/@joonion)
#
#I just changed a few things for qiskit 1.2.4
# *execute -> run
# *cu1 -> cp
 

In [3]:
import math
import random
import numpy as np
from fractions import Fraction
from qiskit import transpile,  QuantumCircuit 
import matplotlib.pyplot as plt
from qiskit_ibm_provider import IBMProvider

def factorize4(N):
    trial = 0
    while (True):
        trial += 1
        print('trial =', trial)
        a = random.randint(2, N - 1)
        if a not in [2, 7, 8, 11, 13]:
            continue
        r, qc = findPeriodByQuantumCircuit(N, a)
        print('\ta =', a, 'r =', r)
        if (r % 2 != 0):
            continue
        gcd1 = math.gcd(N, a ** (r // 2) + 1)
        gcd2 = math.gcd(N, a ** (r // 2) - 1)
        print('\tgcd1 =', gcd1, 'gcd2 =', gcd2)
        if (gcd1 == 1 or gcd2 == 1):
            continue
        return gcd1, gcd2, qc

def findPeriodByQuantumCircuit(N, a):
    phase, qc = qpe_amod15(a)
    frac = Fraction(phase).limit_denominator(15)
    return frac.denominator, qc

def qpe_amod15(a):
    n_count = 3
    qc = QuantumCircuit(4 + n_count, n_count)
    for q in range(n_count):
        qc.h(q) 
    qc.x(3 + n_count) 
    for q in range(n_count): 
        qc.append(c_amod15(a, 2 ** q), [q] + [i + n_count for i in range(4)])
    qc.append(qft_dagger(n_count), range(n_count)) 
    qc.measure(range(n_count), range(n_count))

    provider = IBMProvider()
    backend = provider.get_backend('ibm_brisbane')

 
    qc_t = transpile(qc, backend)
    job = backend.run(qc_t, memory=True)

    result = job.result()
    readings = result.get_memory()
    
    phase = int(readings[0], 2) / (2 ** n_count)
    return phase, qc

def c_amod15(a, power):
    if a not in [2, 7, 8, 11, 13]:
        raise ValueError("'a' must be in [2, 7, 8, 11, 13]")
    U = QuantumCircuit(4)
    for iteration in range(power):
        if a in [2, 13]:
            U.swap(0, 1)
            U.swap(1, 2)
            U.swap(2, 3)
        if a in [7, 8]:
            U.swap(2, 3)
            U.swap(1, 2)
            U.swap(0, 1)
        if a == 11:
            U.swap(1, 3)
            U.swap(0, 2)
        if a in [7, 11, 13]:
            for q in range(4):
                U.x(q)
    U = U.to_gate()
    U.name = " %i^%i mod 15" % (a, power)
    c_U = U.control()
    return c_U

def qft_dagger(n):
    qc = QuantumCircuit(n)
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi / float(2**(j-m)), m, j)
        qc.h(j)
    qc.name = " QFT† (I-QFT)"
    return qc

def main():
    N = 3 * 5
    p, q, qc = factorize4(N)
    print(N, '=', p, '*', q)
    qc.draw()
        
main()

# QPE 회로 그리기
a = 7
phase, qc = qpe_amod15(a)
qc.draw()

# Modular Power 회로 그리기
a = 2
power = 4
U = QuantumCircuit(4)
for iteration in range(power):
    if a in [2, 13]:
        U.swap(0, 1)
        U.swap(1, 2)
        U.swap(2, 3)
    if a in [7, 8]:
        U.swap(2, 3)
        U.swap(1, 2)
        U.swap(0, 1)
    if a == 11:
        U.swap(1, 3)
        U.swap(0, 2)
    if a in [7, 11, 13]:
        for q in range(4):
            U.x(q)
U.draw()

# I-QFT 회로 그리기

qc = qft_dagger(3)
qc.draw()

trial = 1
	a = 13 r = 8
	gcd1 = 1 gcd2 = 15
trial = 2
	a = 11 r = 8
	gcd1 = 1 gcd2 = 15
trial = 3
trial = 4
	a = 13 r = 8
	gcd1 = 1 gcd2 = 15
trial = 5
trial = 6
trial = 7
trial = 8
trial = 9
trial = 10
trial = 11
trial = 12
trial = 13
trial = 14
trial = 15
trial = 16
trial = 17
trial = 18
trial = 19
trial = 20
	a = 11 r = 4
	gcd1 = 1 gcd2 = 15
trial = 21
	a = 11 r = 8
	gcd1 = 1 gcd2 = 15
trial = 22
trial = 23
trial = 24
trial = 25
trial = 26
trial = 27
trial = 28
	a = 2 r = 8
	gcd1 = 1 gcd2 = 15
trial = 29
	a = 2 r = 4
	gcd1 = 5 gcd2 = 3
15 = 5 * 3


┌───┐                                        
q_0: ─X─┤ H ├─■──────────────■───────────────────────
      │ └───┘ │P(-π/2) ┌───┐ │                       
q_1: ─┼───────■────────┤ H ├─┼─────────■─────────────
      │                └───┘ │P(-π/4)  │P(-π/2) ┌───┐
q_2: ─X──────────────────────■─────────■────────┤ H ├
                                                └───┘